In [112]:
import numpy as np
import pandas as pd
import neuralnetworksA2 as nn
import imp
from PIL import Image
import os
import numpy as np
import scipy.misc as smp



def readFiles(directory_str):
    array_of_pictures = []
    directory     = os.fsencode(directory_str)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".png"):
            img = Image.open(directory_str+filename)
            pixel_index = img.load()
            height, width = img.size
            data = np.array(img.getdata())
            picture = []
            for i in range(height):
                row = []
                for j in range(width):
                    row.append(pixel_index[i, j])
                picture.append(row)
            np_picture = np.array(picture)
        if  len(np_picture.shape) == 3 and np_picture.shape[2] == 3 :
            array_of_pictures.append(picture)
            
    return array_of_pictures

def standardizePictures(array_of_pictures):
    # Standardizing pictures so they all have the same dimension<><><>< NOT NEEDED (I think) <><><><><
    for picture in array_of_pictures:
        for row in picture:
            while len(row) < 367:   #Careful when using this line, when using other pictures
                row.append((0,0,0))
    array_of_pictures = [picture for picture in array_of_pictures if np.array(picture).shape == (550, 367, 3)]
    return array_of_pictures
    #     print(np.array(picture).shape)

In [113]:

#Create arrays for training and testing pictures

training_pictures = readFiles('./flags-normal/train-data/')
testing_pictures  = readFiles('./flags-normal/test-data/')

training_pictures = standardizePictures(training_pictures)
testing_pictures = standardizePictures(testing_pictures)



In [114]:
print(np.array(training_pictures).shape)
print(np.array(testing_pictures).shape)

(65, 550, 367, 3)
(4, 550, 367, 3)


In [118]:
netc = nn.NeuralNetwork(882, [10, 10], 219)


for picture in training_pictures[0:4]:
#     picture = np.array(picture).
    # Split up picture
    X = np.array([row[int(0.2 * len(row)):] for row in picture])
    # print(len(X))
    T = np.array([row[0:int(0.2 * len(row))] for row in picture])

#     print("Shapes before reshape")
#     print(X.shape)
#     print(T.shape)
    # print(len(T))
    X_reshaped = X.reshape((X.shape[0], -1))
    T_reshaped= T.reshape((T.shape[0], -1))

#     print("\nShapes after reshape")
#     print(X_reshaped.shape)
#     print(T_reshaped.shape)     

    netc = nn.NeuralNetwork(X_reshaped.shape[1], [1], T_reshaped.shape[1])
    netc.train(X_reshaped, T_reshaped, 100)

In [119]:

for picture in testing_pictures:  # To do a subset of all pictures, specify training_pictures[0:n]
#     picture = np.array(picture).T    # If we wanted to transpose, doesn't look like it works
#     print("Picture dimension" + str(np.array(picture).shape))
    # Split up picture
    X = np.array([row[int(0.2 * len(row)):] for row in picture])
    # print(len(X))
    T = np.array([row[0:int(0.2 * len(row))] for row in picture])
    
#     print("Shapes before reshape")
#     print(X.shape)
#     print(T.shape)
    X_reshaped = X.reshape((X.shape[0], -1))
    T_reshaped= T.reshape((T.shape[0], -1))
    
#     print("\nShapes after reshape")
#     print(X_reshaped.shape)
#     print(T_reshaped.shape)  
    
    predicted_values = netc.use(X_reshaped).reshape(T.shape)

    predict_pic = np.append(predicted_values, X, axis = 1)
    
    img = smp.toimage(predict_pic)       # Create a PIL image
    img.show()


In [21]:
# import numpy as np
# import scipy.misc as smp
# imp.reload(nn)

# picture1 = picture
# # Create a 1024x1024x3 array of 8 bit unsigned integers
# data = np.zeros( (len(picture1[0]),len(picture1[0][0]),3), dtype=np.uint8 )

# for row in range(0,len(picture1[0])):
#     for col in range(0,len(picture1[0][0])):
#         data[row][col] = picture1[0][row][col]

# img = smp.toimage(data)       # Create a PIL image
# img.show()